**Установка библиотек**

Transformers предоставляет API и инструменты для простой загрузки и настройки современных предварительно обученных моделей.

Accelerate - это библиотека, которая позволяет запускать один и тот же основной код на распределенной конфигурации. Нужна чтобы ускорить процессы.

Bitsandbytes предоставляет функции, позволяющие значительно сократить потребление памяти для ответов и обучения. Большинство моделей большинство моделей llm содержат 16 бит, но поскольку мы в Google collab, то будем квантовать их до 4х бит.

In [ ]:
!pip install -q transformers accelerate bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 6.2 MB/s eta 0:00:00


**Создаём объект модели и загружаем веса из предварительно обученной модели с именем "NousResearch/Llama-2-7b-chat-hf"**

AutoModelForCausalLM - это универсальный класс для работы с авторегрессивными моделями. Используем его для создания объекта модели.

from_pretrained() - для загрузки весов из модели.


*   NousResearch/Llama-2-7b-chat-hf - имя модели
*   device_map="auto"- определяет устройство, которое будет использоваться для вычислений. Установка значения "auto" означает, что библиотека автоматически выберет доступное оборудование (CPU или GPU) для вычислений.
*   load_in_4bit=True - определяет, следует ли загружать модель с 4-битной точностью или нет. Загрузка модели с 4-битной точностью позволяет оптимизировать память и ускорить вычисления, особенно на поддерживаемом оборудовании.





In [ ]:
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(
    "NousResearch/Llama-2-7b-chat-hf", device_map="auto", load_in_4bit=True
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/179 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:415: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuratio

**Инициализируем токенизатор и токенизируем инпут (входной текст)**

AutoTokenizer - это универсальный класс токенизатора. Инициализируем объект токенизатора с использованием этого класса и загружаем токенизатор, связанный с предварительно обученной моделью с помощью метода from_pretrained():

*   Nous Research/Llama-2-7b-chat-hf - параметр указывает имя или путь к предварительно обученной модели, токенизатор которой мы будем использовать.

*   padding_side="left" - указывает, с какой стороны следует добавлять заполняющие токены при кодировании последовательностей. В этом случае заполняющие токены будут добавлены в левую часть последовательностей. Обычно заполнение добавляется для того, чтобы все последовательности в пакете имели одинаковую длину.

Далее используем токенизатор для токенизации входного текста "SIES is" и преобразования его в формат, подходящий для ввода в модель:


*   ["SIES is"] - параметр определяет список вводимых текстов, подлежащих токенизации. В этом случае вводимый текст только один: "SIES is".
*   return_tensors="pt" - определяет тип возвращаемых тензоров. Значение "pt" указывает на то, что должны быть возвращены тензоры PyTorch. Другие варианты: тензоры TensorFlow ("tf") или массивы NumPy ("np").


Переменная model_inputs содержит словарь с токенизированным входным текстом, преобразованным в тензоры PyTorch, готовые к использованию в качестве входных данных для модели.


In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("NousResearch/Llama-2-7b-chat-hf", padding_side="left")
model_inputs = tokenizer(["SIES is"], return_tensors="pt")

tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

**Генерируем токены на основе инпута и преобразовываем их в текст. Первая генерация**

model.generate - предварительно обученная языковая модель model генерирует последовательности токенов с помощью метода generate:


*   input_ids - этот параметр определяет входные данные для модели. Он принимает токенизированный инпут, ранее сгенерированный токенизатором, и перемещает их в графический процессор (если он доступен).
** model_inputs["input_ids"] извлекает идентификаторы входных токенов, сгенерированные токенизатором.
** to("cuda") перемещает эти идентификаторы в графический процессор для ускорения вычислений.
*   max_new_tokens - максимальное количество новых токенов, которые должна генерировать модель.
*   temperature - параметр определяет случайность генерации токенов (насколько модель креативна). Более низкие температуры приводят к меньшей случайности, обеспечивая более детерминированные выходные данные.
*   do_sample - указывает, следует ли использовать выборку при генерации токенов. Если установлено значение True, модель будет произвольно выбирать токены из распределения вероятности по всему словарю с различными корректировками, зависящими от конкретной стратегии. В противном случае используется жадное декодирование.
Значение true привносит элемент случайности, привнося творческий подход в выходные данные LLM.

tokenizer.batch_decode: токенизатор расшифровывает сгенерированные идентификаторы токенов обратно в удобочитаемый текст с помощью метода batch_decode.     
Метод принимает идентификаторы токенов, которые необходимо декодировать. Передаём generated_ids - выходные данные метода model.generate.

Поскольку tokenizer.batch_decode возвращает список декодированных последовательностей, используем [0] для обращения к первой из них.

В итоге получаем читаемый текст, сгенерированный на основе входных данных, предоставленных модели.

In [ ]:
generated_ids = model.generate(input_ids=model_inputs["input_ids"].to("cuda"), max_new_tokens=280, temperature=0.1, do_sample=True)
tokenizer.batch_decode(generated_ids)[0]

"<s> SIES is a leading provider of IT and business process outsourcing services. Unterscheidung between IT outsourcing and business process outsourcing. IT outsourcing refers to the practice of contracting with an external provider to manage and maintain a company's information technology infrastructure and systems. This can include tasks such as managing servers, databases, networks, and software applications. Business process outsourcing (BPO), on the other hand, refers to the practice of contracting with an external provider to manage and perform specific business processes, such as customer service, data entry, or financial processing.\n\nSIES provides a range of services, including:\n\n1. IT Outsourcing: SIES provides a comprehensive range of IT outsourcing services, including infrastructure management, application management, and security management.\n2. BPO: SIES offers a wide range of BPO services, including customer service, data entry, and financial processing.\n3. Cloud Serv

**Вторая генерация, другой инпут**

In [ ]:
model_inputs = tokenizer(["ViPNet SIES is"], return_tensors="pt")
generated_ids = model.generate(input_ids=model_inputs["input_ids"].to("cuda"), max_new_tokens=280, temperature=0.1, do_sample=True)
tokenizer.batch_decode(generated_ids)[0]

'<s> ViPNet SIES is a network of 150+ institutions in 50+ countries, working together to advance the field of visual perception and its applications. Unterscheidung zwischen "Sehen" und "Wahrnehmen" ist wichtig, da sie verschiedene Aspekte des visuellen Erlebens beschreiben. Visual Perception is a multidisciplinary field that draws on psychology, neuroscience, computer science, and philosophy to understand how we perceive and interpret visual information from the world around us. The study of visual perception involves understanding how the brain processes visual information from the retina, how it interprets and makes sense of visual stimuli, and how it uses visual information to guide behavior and cognition. Visual perception is a complex process that involves multiple stages, from the initial detection of visual stimuli to the interpretation and understanding of what is being seen. The field of visual perception is constantly evolving, with new research and technologies shedding lig

**Третья генерация, тот же инпут, другие параметры**

In [ ]:
model_inputs = tokenizer(["ViPNet SIES is"], return_tensors="pt")
generated_ids = model.generate(input_ids=model_inputs["input_ids"].to("cuda"), max_new_tokens=280)
tokenizer.batch_decode(generated_ids)[0]

'<s> ViPNet SIES is a software tool for the analysis of vascular plants (vascular flora) and their ecological relevance. Unterscheidung von Wild- und Cultivarbäumen. The software is designed to help botanists and ecologists to identify and analyze vascular plants, and to distinguish between wild and cultivated individuals.\n\nViPNet SIES is based on a database of over 100,000 images of vascular plants, which are organized into a hierarchical system of taxa and subtaxa. The software uses machine learning algorithms to identify the plant species in an image, and to determine its taxonomic rank. The software can also be used to analyze the ecological relevance of vascular plants, such as their role in shaping ecosystems, their interactions with other organisms, and their responses to environmental changes.\n\nViPNet SIES is a powerful tool for the analysis of vascular plants, and it has many applications in ecology, conservation, and horticulture. For example, the software can be used to:

**Четвёртая генерация, другой инпут**

In [ ]:
model_inputs = tokenizer(["SIES Workstations is used for"], return_tensors="pt")
generated_ids = model.generate(input_ids=model_inputs["input_ids"].to("cuda"), max_new_tokens=280)
tokenizer.batch_decode(generated_ids)[0]

'<s> SIES Workstations is used for a wide range of applications, including:\n Unterscheidung von Daten in verschiedenen Formaten: SIES Workstations supports the conversion of data between different formats, such as text, images, audio, and video. This makes it easy to work with data from different sources and in different formats.\n\nData Encryption and Decryption: SIES Workstations provides advanced data encryption and decryption capabilities, ensuring that sensitive data is protected from unauthorized access.\n\nData Compression: SIES Workstations supports data compression, which helps to reduce the size of data files and improve data transfer efficiency.\n\nData Backup and Recovery: SIES Workstations provides advanced data backup and recovery capabilities, ensuring that data is safely stored and easily recovered in the event of a system failure or data loss.\n\nData Mining and Analytics: SIES Workstations supports advanced data mining and analytics capabilities, enabling users to ex

**Пятая и шестая генерации, общие инпуты**

In [ ]:
model_inputs = tokenizer(["New York is"], return_tensors="pt")
generated_ids = model.generate(input_ids=model_inputs["input_ids"].to("cuda"), max_new_tokens=280)
tokenizer.batch_decode(generated_ids)[0]

"<s> New York is a state in the northeastern United States. Unterscheidung zwischen den verschiedenen Arten von New York City, including Manhattan, Brooklyn, Queens, the Bronx, and Staten Island.\n\nNew York City is the most populous city in the United States and consists of five boroughs: Manhattan, Brooklyn, Queens, the Bronx, and Staten Island. Each borough has its own unique culture, history, and attractions.\n\nManhattan is the most populous borough and is home to many of New York City's most famous landmarks, including Times Square, Central Park, and the Empire State Building. Brooklyn is known for its vibrant arts scene, diverse neighborhoods, and trendy restaurants. Queens is the most ethnically diverse county in the United States and is home to many international cuisines and cultural attractions. The Bronx is home to the New York Yankees baseball team and has a rich history in hip-hop and rap music. Staten Island is the least populous borough and is known for its suburban atm

In [ ]:
model_inputs = tokenizer(["To cook fried potatoes"], return_tensors="pt")
generated_ids = model.generate(input_ids=model_inputs["input_ids"].to("cuda"), max_new_tokens=280)
tokenizer.batch_decode(generated_ids)[0]

'<s> To cook fried potatoes, you will need the following ingredients:\n Unterscheidung zwischen "Fried Potatoes" und "Baked Potatoes" - YouTube.\nFried potatoes are a popular side dish that can be prepared in a variety of ways. Here are some common methods for cooking fried potatoes:\n\n1. Pan-frying: This is the most common method for cooking fried potatoes. Simply slice the potatoes into thin rounds or wedges, place them in a pan with a small amount of oil, and cook over medium-high heat until they are golden brown and crispy.\n2. Deep-frying: For a crisper, more flavorful fried potato, try deep-frying them in hot oil. This method involves submerging the potatoes in hot oil in a deep fryer or a large pot with a thermometer. Cook the potatoes until they are golden brown and crispy, then remove them from the oil with a slotted spoon and drain on paper towels.\n3. Oven-frying: This method involves baking the potatoes in the oven instead of frying them in oil. Simply slice the potatoes i